## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-26-22-49-00 +0000,wsj,Protected Status for Haitian Migrants to Expir...,https://www.wsj.com/politics/policy/protected-...
1,2025-11-26-22-48-26 +0000,nyt,North Carolina Can Use Republican-Friendly Con...,https://www.nytimes.com/2025/11/26/us/north-ca...
2,2025-11-26-22-48-01 +0000,nyt,Live Updates: Fire Engulfs Hong Kong Apartment...,https://www.nytimes.com/live/2025/11/26/world/...
3,2025-11-26-22-47-32 +0000,wapo,"Fire engulfs high-rise buildings in Hong Kong,...",https://www.washingtonpost.com/world/2025/11/2...
4,2025-11-26-22-44-59 +0000,nyt,"As Trade Talks Stall, Carney Moves to Shield C...",https://www.nytimes.com/2025/11/26/world/canad...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,42
16,fire,19
202,year,19
19,kong,16
18,hong,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
224,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...,129
171,2025-11-26-16-23-23 +0000,cbc,New prosecutor won't pursue charges against Tr...,https://www.cbc.ca/news/world/trump-georgia-el...,99
175,2025-11-26-16-01-38 +0000,nypost,Georgia drops 2020 election case against Trump...,https://nypost.com/2025/11/26/us-news/peter-sk...,91
73,2025-11-26-20-37-11 +0000,nypost,Trump vows ‘animal’ who shot 2 National Guards...,https://nypost.com/2025/11/26/us-news/trump-an...,91
232,2025-11-26-11-10-35 +0000,nypost,FDA chief promises ‘powerful’ new treatments f...,https://nypost.com/2025/11/26/us-news/fda-chie...,89


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
224,129,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...
21,82,2025-11-26-22-14-20 +0000,bbc,At least 44 dead and hundreds missing after fi...,https://www.bbc.com/news/articles/cvg165d7ppvo...
171,57,2025-11-26-16-23-23 +0000,cbc,New prosecutor won't pursue charges against Tr...,https://www.cbc.ca/news/world/trump-georgia-el...
219,53,2025-11-26-12-02-40 +0000,nypost,White House blames Democrat-led states for ‘dr...,https://nypost.com/2025/11/26/us-news/white-ho...
76,44,2025-11-26-20-33-00 +0000,wsj,The job market continues to sputter even as co...,https://www.wsj.com/economy/central-banking/fe...
230,43,2025-11-26-11-13-21 +0000,nypost,Marty Makary touts FDA efforts to crack down o...,https://nypost.com/2025/11/26/us-news/marty-ma...
145,43,2025-11-26-17-31-50 +0000,nyt,Trump Defends Witkoff’s Tactics in Russia-Ukra...,https://www.nytimes.com/2025/11/26/us/politics...
289,36,2025-11-26-07-48-43 +0000,missionlocal,Federal court orders ICE to provide mattresses...,https://missionlocal.org/2025/11/federal-court...
221,32,2025-11-26-12-00-00 +0000,startribune,"In Minnesota farm country, a plan for a $4 bil...",https://www.startribune.com/in-minnesota-farm-...
35,32,2025-11-26-21-54-06 +0000,nypost,Ryan Lizza claims then-fiancée Olivia Nuzzi we...,https://nypost.com/2025/11/26/us-news/ryan-liz...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
